In [ ]:
!nvidia-smi

Wed May 26 09:46:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q git+https://github.com/eaedk/testing-zindi-package.git

In [ ]:
# Import Zindian class from the package
from zindi.user import Zindian

In [ ]:
# login to your Zindi account to download the dataset
my_username = "your username"
user = Zindian(username = my_username)

In [ ]:
user.select_a_challenge()                               # Select a Zindi challenge
user.which_challenge                                    # Get information about the selected challenge

In [ ]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

dataset/Test.csv: 100%|██████████| 215k/215k [00:00<00:00, 527ko/s] 
dataset/SampleSubmission.csv: 100%|██████████| 200k/200k [00:00<00:00, 359ko/s]
dataset/clips.zip: 100%|██████████| 241M/241M [00:34<00:00, 7.26Mo/s]
dataset/Train.csv: 100%|██████████| 1.05M/1.05M [00:00<00:00, 1.28Mo/s]


In [ ]:
!unzip dataset/clips.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# Create a directory named wolof to save your model

output_models_dir = "gdrive/MyDrive/wolof"

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install jiwer


In [ ]:
import pandas as pd

data = pd.read_csv('dataset/Train.csv')
data['ID'] = 'clips/' + data['ID'] + '.mp3'
data.to_csv('train.csv',index=False)
data.head()


,ID,up_votes,down_votes,age,gender,transcription
0,clips/002e50c29ac2890c7cb3b3d63dcbe512bc6850de...,2,0,NaN,NaN,Malicka
1,clips/0031672b4484f963c8a07babe6f713dd559539d4...,2,0,twenties,female,Ecole Elementaire Pikine Nord
2,clips/00362ccc6b48d3ea225e12ddf8a06a9d582cccc0...,2,0,twenties,male,Cimetière de Cambérène
3,clips/0042cdb4d4a015cddacc26d88faffdd714b7a272...,2,0,twenties,male,Tournalou Boune
4,clips/00439f02fa8f0dade934bdb317199b71662e9396...,2,0,twenties,male,Pharmacie Golf Sud


In [ ]:
# Create training folds(5 folds)

from datasets import load_dataset, load_metric,concatenate_datasets



vals_ds = load_dataset('csv',data_files=['train.csv'], split=[f'train[{k}%:{k+20}%]' for k in range(0, 100, 20)])


voice_val = vals_ds[0].remove_columns(['up_votes', 'down_votes', 'age', 'gender'])


trains_ds = load_dataset('csv',data_files=['train.csv'], split=[f'train[:{k}%]+train[{k+20}%:]' for k in range(0, 100, 20)])

voice_train = trains_ds[0].remove_columns(['up_votes', 'down_votes', 'age', 'gender'])




Using custom data configuration default-949450ca81edf56a


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-949450ca81edf56a/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


Using custom data configuration default-949450ca81edf56a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-949450ca81edf56a/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
# Display the dataset

show_random_elements(voice_train, num_examples=20)

,ID,transcription
0,clips/ec54a7683a9f8644df86624a79ef6543a2313cf5ffeb7318627e1ed4c1c561132e8511c4b234023801000cca9c7d7698663ab857def672fc4a936114c159df92.mp3,ban oto mooy jaar Poste Courant
1,clips/3dfa7452f481c7b78b1d730eb23f431987017da1dbe63aa82dffd5a93ad5eb13fa4028cd5695c74a12e89d1036d554686aab23716a259d38955cb10901251c7d.mp3,Centre Sportiff de la BCEAO
2,clips/e3b2c5d4c288ee030919c4ed687d82a24472501547afcb7460c62ed054d2e9c7d62def2225962b2c9c4c2a566bb3555dce3fbe976f0f6837ea43a9b2e6444d65.mp3,Mosquée Terminus
3,clips/6918a57516f06d8db06b2f803770cc52d05a0cc257bfd6463494e62e47a7cfde174ea3775e8ecdde459ff780af35061b6a7cf6c4a2b61bdd039808ba75c8d106.mp3,Stade Amadou Barry
4,clips/70d180a9ee3f674465f1dc812e56d0c7fff54ec1b85ab33f5319e66dd52632528a2f3effd8311a5785c3b19adc464149f9a42e953e53170f8a8b02cb1875c45b.mp3,Bargny
5,clips/3f1b32e99713ee86bb0026a721e4c08317e4c10d777b12b2eb0040214251aebad91ecb22fd6f3bc4db20b95dbce1fa67c48b9ce8b5ed7d0e4374034ee3d0bc73.mp3,Centre de Sauvegarde
6,clips/c17d72db1542780d4263888758ed21bd34a73c85c503a33613e89187cd0eea5b68b3694ea7c42e4a6efc12fb3155c79d7fea8205f78a8d1d73a1b547ab924aed.mp3,Place de l’indépendance
7,clips/8e0a3887012c23492f43abf772f052a018ec56f64c29105739cf140325877113a21cd136ed8a0935f6572eed3d5cbf6e10661c0233b4576714ba9c894cec800f.mp3,Cimetiére Saint Lazzare
8,clips/bfcb5d267cdb2e6f2e425ab018c2e5a222d774d65798e29ec0afabafb9efad102bec40579715890c8dd8894ac4a5ef11d937760382eb3a6f72ab624fdcf6f00b.mp3,Terminus Gouy Mouride Rufisque
9,clips/577877b0b0744412da9d90ea4c3b82add72489fa840d6a9f8c4bf687a60369a8d3fc3197266cffb8bfdbf199eb4c7841d65331d7e5c5cae4882fadf60bb452db.mp3,Cité Pénitence


In [ ]:
import re
chars_to_ignore_regex = '[\,\؟\.\!\-\;\:\"\)\(\«\»\؛\—\ـ\_\،\“\%\‘\”]'

def remove_special_characters(batch):  
    batch["transcription"] = re.sub(chars_to_ignore_regex,'', batch["transcription"]).lower() + " "
    return batch


voice_train = voice_train.map(remove_special_characters)
voice_val = voice_val.map(remove_special_characters)    

In [ ]:
# Display cleaned data

show_random_elements(voice_train, num_examples=20)

,ID,transcription
0,clips/8c3347001da20a744cb3bf6ce1bb532b961970e69ab7951503382fc13a8adaaf30b3d3c5311c27718e926d7ab4884e0365442d376d8349ea014334290a06bf44.mp3,flag diarri poul
1,clips/ce3710a710a352a856b82f9eacaca62e08bc63649b1e1394a4a9b6a591c35766e4823bcdec3378ae5843395abeffa09e60c39d4e3a308e94c1e275f34479528f.mp3,avenue cheikh anta diop
2,clips/3ffe6832d7bff6363826a83f04dd01df7b6ef55746afd5b021f062c332ce9a7c1937bf054fbea6aa1b233bb80ce2531aad6fa4b6dc113dce66575bea8c77df58.mp3,bén tally
3,clips/9ff1324a4decdefe61b427ecea5f8708855195c26483c8e7b93ea368bc13fbb27bb2e6864903b4ebeda7566ba9924af034e6234022146ad547da8d550ecd854c.mp3,pharmacie mamelle
4,clips/70d180a9ee3f674465f1dc812e56d0c7fff54ec1b85ab33f5319e66dd52632528a2f3effd8311a5785c3b19adc464149f9a42e953e53170f8a8b02cb1875c45b.mp3,bargny
5,clips/5096405ea10267d42961eded7e687cdf42a797136b70dac8cfc33cac910d586d11a89a45bfc60fd3a56e75ead91f89e9e225d04821480fa345236d12691c6da8.mp3,place du souvenir africain
6,clips/ddcdba0abefab8198bb3cdd2ff7e08125e5ba4d00e09f459d0080de663aad4a70074a9a4d941ee8f5acbb2000b16d652bc1c6fd982556dda5e6f47ad4269548e.mp3,thierno ndiaye
7,clips/d55029c146abb91c0f85cf87da75aec183f095ee3d2e7f775e81f2fa7925f0caacfd684d6314c92cc655ac5c2efc2f4f9738cd91242133ad6248d43c01261c6d.mp3,parc forestier de hann
8,clips/357ac99aa10c2b4d11e11bfc1efcdc42e0e4d2f2dba57c3b7bd8475597ea376d459d7853c95ff0262b7efbfd00ed7514124711ef6c56f0c6faf46c62aa7a73a8.mp3,orange médina
9,clips/9ddb3edd7d9cb84e0fca8ac51f0be1e32478c50c1b212fbffc7338cc0fb7e4c36679ac604539a1a6899538d997e36c28bb3a5be9fd7ad6615ceca0f5ffeb091d.mp3,cité douane golf sud océan


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
vocab_train = voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=voice_train.column_names)
vocab_val = voice_val.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=voice_val.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_val["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 25,
 "'": 1,
 'a': 19,
 'b': 24,
 'c': 29,
 'd': 30,
 'e': 0,
 'f': 5,
 'g': 21,
 'h': 36,
 'i': 9,
 'j': 13,
 'k': 10,
 'l': 27,
 'm': 22,
 'n': 18,
 'o': 34,
 'p': 12,
 'q': 3,
 'r': 23,
 's': 16,
 't': 32,
 'u': 26,
 'v': 6,
 'w': 15,
 'x': 28,
 'y': 7,
 'z': 2,
 'â': 17,
 'ç': 33,
 'è': 35,
 'é': 11,
 'ë': 31,
 'î': 4,
 'ô': 14,
 'œ': 8,
 '’': 20}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

39

In [ ]:
# Prepare vocabulary
import json
with open(f"{output_models_dir}/vocab.json", 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# Tokenization
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(f"{output_models_dir}/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained(output_models_dir)

In [ ]:
voice_train[0]

{'ID': 'clips/3344aa00fd1aac2f570fc6aa918e522e6ab4f6306cd12a8b18300ad062d6acca99ffffa6783e9569ea15b19e364393a45c66d8ad11c69bee7183500f009643ae.mp3',
 'transcription': 'terrain mbedou fass '}

In [ ]:
import torchaudio
import torch

# Apply transformation to 0.5 of the training data(reduce speed + reverbration)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["ID"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch

def speech_file_to_array_fn_aug(batch):
    speech_array, sampling_rate = torchaudio.load(batch["ID"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    #batch["speech"] = resampler(speech_array).squeeze().numpy()
    effects = [
           ["lowpass", "-1", "300"], # apply single-pole lowpass filter
           ["speed", "0.9"],  # reduce the speed
                     # This only changes sample rate, so it is necessary to
                     # add `rate` effect with original sample rate after this.
           ["rate", f"{sampling_rate}"],
           #["reverb", "-w"],  # Reverbration gives some dramatic feeling
           ]
    effects2 = [["reverb","-w"]]

    # Apply effects
    prob = torch.rand(1)
    rever = torch.rand(1)
    if prob > 0.5:
      waveform2, sample_rate2 = torchaudio.sox_effects.apply_effects_tensor(speech_array, sampling_rate, effects) 
      batch["speech"] = resampler(waveform2).squeeze().numpy()
      batch["sampling_rate"] = 16000
      batch["target_text"] = batch["transcription"]
    if rever>0.5:
      waveform3, sample_rate3 = torchaudio.sox_effects.apply_effects_tensor(speech_array, sampling_rate, effects2) 
      batch["speech"] = resampler(waveform3).squeeze().numpy()
      batch["sampling_rate"] = 16000
      batch["target_text"] = batch["transcription"]  


    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["transcription"]
    return batch


In [ ]:
voice_train = voice_train.map(speech_file_to_array_fn_aug, remove_columns=voice_train.column_names,num_proc=1)
voice_val = voice_val.map(speech_file_to_array_fn, remove_columns=voice_val.column_names,num_proc=1)

In [ ]:
import IPython.display as ipd
import numpy as np
import random
# Play audio
rand_int = random.randint(0, len(voice_train)-1)

ipd.Audio(data=np.asarray(voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(voice_train)-1)

print("Target text:", voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", voice_train[rand_int]["sampling_rate"])

Target text: cem hann 
Input array shape: (33792,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
voice_train = voice_train.map(prepare_dataset, remove_columns=voice_train.column_names, batch_size=8, num_proc=4, batched=True)
voice_val = voice_val.map(prepare_dataset, remove_columns=voice_val.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.05,
    hidden_dropout=0.05,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=output_models_dir,
  #output_dir="dev/",
  group_by_length=True,
  per_device_train_batch_size=16,
  #per_device_eval_batch_size=32,
  gradient_accumulation_steps=2,
  dataloader_num_workers = 1,
  evaluation_strategy="steps",
  num_train_epochs=51,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=1,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=voice_train,
    eval_dataset=voice_val,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train() 

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Inference on test data

test_dataset = load_dataset('csv',data_files=['dataset/SampleSubmission.csv'])



processor = Wav2Vec2Processor.from_pretrained(output_models_dir)
model = Wav2Vec2ForCTC.from_pretrained(f"{output_models_dir}/checkpoint-4000")
model.to("cuda")


# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(f"clips/{batch['ID']}.mp3")
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch


test_dataset = test_dataset.map(speech_file_to_array_fn)

def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
         logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["transcription"] = processor.batch_decode(pred_ids)
    return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("Prediction:", result)

Using custom data configuration default-eebb785c5c9169c0
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-eebb785c5c9169c0/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



Prediction: DatasetDict({
    train: Dataset({
        features: ['ID', 'speech', 'transcription'],
        num_rows: 1564
    })
})


In [ ]:
import pandas as pd


sub = pd.read_csv('dataset/SampleSubmission.csv')
sub.head()


,ID,transcription
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,0
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,0
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,0
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,0
4,01337d7daaf50c9a53b783f4cd6704b5a152dd5d299899...,0


In [ ]:
result

DatasetDict({
    train: Dataset({
        features: ['ID', 'speech', 'transcription'],
        num_rows: 1564
    })
})

In [ ]:
# fill the empty audio with None
final_pred = [ 'None' if pred=='' else pred for pred in result['train']['transcription']]
final_pred[1480]
        

'None'

In [ ]:
sub['transcription'] =  final_pred
sub.to_csv('submission.csv',index=False)
sub.head(10)

,ID,transcription
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,africatel avss
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,nan laay def ngir dem tally bou bess
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,africatel avs
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,mosquée de cambérène
4,01337d7daaf50c9a53b783f4cd6704b5a152dd5d299899...,ecole hlm rufisque
5,016a99acf910b17ae7e2780ae71884c71dea8eeb1d4a51...,rue baffa séne
6,01eb602b69c303a54315a272c3544daba1eb3a3341b1ec...,ecole primaire privée les petits guénies
7,0235405604f73d5ba9500dc44c293e6ebcb7575f830c47...,marché ouakam
8,0266e6e8f354f4a241b744a64a226d24b90aeef8160ec8...,dieupeul derklé
9,027d12b2a3bd049c026d2103b849de47076a785f76c174...,bargny rails


In [ ]:
import IPython.display as ipd
import numpy as np

speech_array, sampling_rate = torchaudio.load(f"clips/{sub['ID'][4]}.mp3")
resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
batch = resampler(speech_array).squeeze().numpy()
ipd.Audio(data=np.asarray(batch), autoplay=True, rate=16000)